<a href="https://colab.research.google.com/github/Aryamuda/Stock-and-Performance-Visualizer-/blob/main/5_Stock_And_Comparison_Performance_With_ETF_SNP500_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AAPL

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker_symbol, period, interval):
    stock = yf.Ticker(ticker_symbol)
    hist_data = stock.history(period=period, interval=interval)

    return hist_data

ticker_symbol = "AAPL"  # Ticker
data_period = "5y"
data_interval = "1d"

# Fetching
aapl = fetch_stock_data(ticker_symbol, period=data_period, interval=data_interval)

## APPLYING TECHNICAL INDICATOR

In [ ]:
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

sma_20 = SMAIndicator(close=aapl["Close"], window=20, fillna=True)
aapl['SMA_20'] = sma_20.sma_indicator()

sma_50 = SMAIndicator(close=aapl["Close"], window=50, fillna=True)
aapl['SMA_50'] = sma_50.sma_indicator()

rsi_14 = RSIIndicator(close=aapl["Close"], window=14, fillna=True)
aapl['RSI_14'] = rsi_14.rsi()

vwap_indicator = VolumeWeightedAveragePrice(high=aapl["High"],
                                                low=aapl["Low"],
                                                close=aapl["Close"],
                                                volume=aapl["Volume"],
                                                window=14,
                                                fillna=True)
aapl['VWAP_14'] = vwap_indicator.volume_weighted_average_price()

## PLOT AAPL

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

required_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20', 'SMA_50', 'RSI_14', 'VWAP_14', 'Dividends']
if 'stock_df' in locals() and not aapl.empty and all(col in aapl.columns for col in required_cols):

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, # Space between RSI and main chart
                        row_heights=[0.7, 0.3],
                        specs=[[{"secondary_y": True}], # Main chart with secondary y-axis for volume
                               [{"secondary_y": False}]] # RSI chart
                      )

    # 1. Candlestick Chart
    fig.add_trace(go.Candlestick(x=aapl.index,
                                 open=aapl['Open'],
                                 high=aapl['High'],
                                 low=aapl['Low'],
                                 close=aapl['Close'],
                                 name='Candlestick'),
                  row=1, col=1, secondary_y=False)

    # 2. Volume Bars
    fig.add_trace(go.Bar(x=aapl.index, y=aapl['Volume'], name='Volume',
                         marker_color='rgba(128,128,128,0.5)'), # Grey color for volume
                  row=1, col=1, secondary_y=True)

    # 3. Add Technical Indicators to Chart
    color_sma20 = 'blue'
    color_sma50 = 'orange'
    color_vwap14 = 'green'
    color_rsi14 = 'brown'
    color_dividend = 'purple' # Color for dividend markers

    fig.add_trace(go.Scatter(x=aapl.index, y=aapl['SMA_20'], mode='lines',
                             line=dict(color=color_sma20, width=1.5), name='SMA 20'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=aapl.index, y=aapl['SMA_50'], mode='lines',
                             line=dict(color=color_sma50, width=1.5), name='SMA 50'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=aapl.index, y=aapl['VWAP_14'], mode='lines',
                             line=dict(color=color_vwap14, width=1.5), name='VWAP 14'),
                  row=1, col=1, secondary_y=False)

    # 4. Plot Dividends on Main Chart
    dividends_df = aapl[aapl['Dividends'] > 0]
    if not dividends_df.empty:
        price_range_for_offset = aapl['High'].max() - aapl['Low'].min()
        if price_range_for_offset == 0: price_range_for_offset = aapl['Low'].mean()
        dividend_y_position = dividends_df['Low'] - (price_range_for_offset * 0.03)

        fig.add_trace(go.Scatter(
            x=dividends_df.index,
            y=dividend_y_position,
            mode='markers',
            marker=dict(
                symbol='triangle-down', # Symbol for dividend
                size=10,
                color=color_dividend,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name='Dividends',
            text=[f'Dividend: {amt:.2f}' for amt in dividends_df['Dividends']],
            hoverinfo='text+x'
        ), row=1, col=1, secondary_y=False)

    # RSI
    fig.add_trace(go.Scatter(x=aapl.index, y=aapl['RSI_14'], mode='lines',
                             line=dict(color=color_rsi14, width=1.5), name='RSI 14'),
                  row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="rgba(255,0,0,0.7)", line_width=1, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="rgba(0,128,0,0.7)", line_width=1, row=2, col=1)


    # Update Layout
    fig.update_layout(
        title=f'{ticker_symbol} - Stock Price, Indicators, and Dividends', # Updated title
        xaxis_title='Date',
        yaxis_title='Price ($)',
        legend_title_text='Legend',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False,
        height=700,
        hovermode='x unified'
    )

    fig.update_yaxes(title_text="Price ($)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Volume", row=1, col=1, secondary_y=True, showticklabels=True,
                     range=[0, aapl['Volume'].max() * 4])
    fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=False)

    fig.show()


elif 'aapl' in locals() and not all(col in aapl.columns for col in required_cols):
    missing_cols = [col for col in required_cols if col not in aapl.columns]
    print(f"Missing required column(s) for plotting:")
    print(f"Available columns in aapl: {aapl.columns.tolist()}")
    if 'Dividends' not in aapl.columns:
        print("Note: 'Dividends' column is missing.")
else:
    print("Stock DataFrame ('aapl') with required indicators not found or is empty.")
    if 'aapl' in locals():
        print(f"Available columns: {aapl.columns.tolist()}")

# MSFT

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker_symbol, period, interval):
    print(f"Fetching data for {ticker_symbol} for the period: {period} with interval: {interval}...")
    stock = yf.Ticker(ticker_symbol)
    hist_data = stock.history(period=period, interval=interval)

    return hist_data

ticker_symbol = "MSFT"  # Ticker
data_period = "2y"
data_interval = "1d"

# Fetching
msft = fetch_stock_data(ticker_symbol, period=data_period, interval=data_interval)

Fetching data for MSFT for the period: 2y with interval: 1d...


## APPLYING TECHNICAL INDICATOR

In [ ]:
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

sma_20 = SMAIndicator(close=msft["Close"], window=20, fillna=True)
msft['SMA_20'] = sma_20.sma_indicator()

sma_50 = SMAIndicator(close=msft["Close"], window=50, fillna=True)
msft['SMA_50'] = sma_50.sma_indicator()

rsi_14 = RSIIndicator(close=msft["Close"], window=14, fillna=True)
msft['RSI_14'] = rsi_14.rsi()

vwap_indicator = VolumeWeightedAveragePrice(high=msft["High"],
                                                low=msft["Low"],
                                                close=msft["Close"],
                                                volume=msft["Volume"],
                                                window=14,
                                                fillna=True)
msft['VWAP_14'] = vwap_indicator.volume_weighted_average_price()

## PLOT MSFT

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

required_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20', 'SMA_50', 'RSI_14', 'VWAP_14', 'Dividends']
if 'msft' in locals() and not msft.empty and all(col in msft.columns for col in required_cols):

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, # Space between RSI and main chart
                        row_heights=[0.7, 0.3],
                        specs=[[{"secondary_y": True}], # Main chart with secondary y-axis for volume
                               [{"secondary_y": False}]] # RSI chart
                      )

    # 1. Candlestick Chart
    fig.add_trace(go.Candlestick(x=msft.index,
                                 open=msft['Open'],
                                 high=msft['High'],
                                 low=msft['Low'],
                                 close=msft['Close'],
                                 name='Candlestick'),
                  row=1, col=1, secondary_y=False)

    # 2. Volume Bars
    fig.add_trace(go.Bar(x=msft.index, y=msft['Volume'], name='Volume',
                         marker_color='rgba(128,128,128,0.5)'), # Grey color for volume
                  row=1, col=1, secondary_y=True)

    # 3. Add Technical Indicators to Chart
    color_sma20 = 'blue'
    color_sma50 = 'orange'
    color_vwap14 = 'green'
    color_rsi14 = 'brown'
    color_dividend = 'purple' # Color for dividend markers

    fig.add_trace(go.Scatter(x=msft.index, y=msft['SMA_20'], mode='lines',
                             line=dict(color=color_sma20, width=1.5), name='SMA 20'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=msft.index, y=msft['SMA_50'], mode='lines',
                             line=dict(color=color_sma50, width=1.5), name='SMA 50'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=msft.index, y=msft['VWAP_14'], mode='lines',
                             line=dict(color=color_vwap14, width=1.5), name='VWAP 14'),
                  row=1, col=1, secondary_y=False)

    # 4. Plot Dividends on Main Chart
    dividends_df = msft[msft['Dividends'] > 0]
    if not dividends_df.empty:
        price_range_for_offset = msft['High'].max() - msft['Low'].min()
        if price_range_for_offset == 0: price_range_for_offset = msft['Low'].mean()
        dividend_y_position = dividends_df['Low'] - (price_range_for_offset * 0.03)

        fig.add_trace(go.Scatter(
            x=dividends_df.index,
            y=dividend_y_position,
            mode='markers',
            marker=dict(
                symbol='triangle-down', # Symbol for dividend
                size=10,
                color=color_dividend,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name='Dividends',
            text=[f'Dividend: {amt:.2f}' for amt in dividends_df['Dividends']],
            hoverinfo='text+x'
        ), row=1, col=1, secondary_y=False)

    # RSI
    fig.add_trace(go.Scatter(x=msft.index, y=msft['RSI_14'], mode='lines',
                             line=dict(color=color_rsi14, width=1.5), name='RSI 14'),
                  row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="rgba(255,0,0,0.7)", line_width=1, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="rgba(0,128,0,0.7)", line_width=1, row=2, col=1)


    # Update Layout
    fig.update_layout(
        title=f'{ticker_symbol} - Stock Price, Indicators, and Dividends', # Updated title
        xaxis_title='Date',
        yaxis_title='Price ($)',
        legend_title_text='Legend',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False,
        height=700,
        hovermode='x unified'
    )

    fig.update_yaxes(title_text="Price ($)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Volume", row=1, col=1, secondary_y=True, showticklabels=True,
                     range=[0, msft['Volume'].max() * 4])
    fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=False)

    fig.show()


elif 'msft' in locals() and not all(col in msft.columns for col in required_cols):
    missing_cols = [col for col in required_cols if col not in msft.columns]
    print(f"Missing required column(s) for plotting:")
    print(f"Available columns in msft: {msft.columns.tolist()}")
    if 'Dividends' not in msft.columns:
        print("Note: 'Dividends' column is missing.")
else:
    print("Stock DataFrame ('msft') with required indicators not found or is empty.")
    if 'msft' in locals():
        print(f"Available columns: {msft.columns.tolist()}")

# NVDA

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker_symbol, period, interval):
    print(f"Fetching data for {ticker_symbol} for the period: {period} with interval: {interval}...")
    stock = yf.Ticker(ticker_symbol)
    hist_data = stock.history(period=period, interval=interval)

    return hist_data

ticker_symbol = "NVDA"  # Ticker
data_period = "2y"
data_interval = "1d"

# Fetching
nvda = fetch_stock_data(ticker_symbol, period=data_period, interval=data_interval)

Fetching data for NVDA for the period: 2y with interval: 1d...


## APPLYING TECHNICAL INDICATOR

In [ ]:
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

sma_20 = SMAIndicator(close=nvda["Close"], window=20, fillna=True)
nvda['SMA_20'] = sma_20.sma_indicator()

sma_50 = SMAIndicator(close=nvda["Close"], window=50, fillna=True)
nvda['SMA_50'] = sma_50.sma_indicator()

rsi_14 = RSIIndicator(close=nvda["Close"], window=14, fillna=True)
nvda['RSI_14'] = rsi_14.rsi()

vwap_indicator = VolumeWeightedAveragePrice(high=nvda["High"],
                                                low=nvda["Low"],
                                                close=nvda["Close"],
                                                volume=nvda["Volume"],
                                                window=14,
                                                fillna=True)
nvda['VWAP_14'] = vwap_indicator.volume_weighted_average_price()

## PLOT NVDA

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

required_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20', 'SMA_50', 'RSI_14', 'VWAP_14', 'Dividends']
if 'nvda' in locals() and not nvda.empty and all(col in nvda.columns for col in required_cols):

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, # Space between RSI and main chart
                        row_heights=[0.7, 0.3],
                        specs=[[{"secondary_y": True}], # Main chart with secondary y-axis for volume
                               [{"secondary_y": False}]] # RSI chart
                      )

    # 1. Candlestick Chart
    fig.add_trace(go.Candlestick(x=nvda.index,
                                 open=nvda['Open'],
                                 high=nvda['High'],
                                 low=nvda['Low'],
                                 close=nvda['Close'],
                                 name='Candlestick'),
                  row=1, col=1, secondary_y=False)

    # 2. Volume Bars
    fig.add_trace(go.Bar(x=nvda.index, y=nvda['Volume'], name='Volume',
                         marker_color='rgba(128,128,128,0.5)'), # Grey color for volume
                  row=1, col=1, secondary_y=True)

    # 3. Add Technical Indicators to Chart
    color_sma20 = 'blue'
    color_sma50 = 'orange'
    color_vwap14 = 'green'
    color_rsi14 = 'brown'
    color_dividend = 'purple' # Color for dividend markers

    fig.add_trace(go.Scatter(x=nvda.index, y=nvda['SMA_20'], mode='lines',
                             line=dict(color=color_sma20, width=1.5), name='SMA 20'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=nvda.index, y=nvda['SMA_50'], mode='lines',
                             line=dict(color=color_sma50, width=1.5), name='SMA 50'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=nvda.index, y=nvda['VWAP_14'], mode='lines',
                             line=dict(color=color_vwap14, width=1.5), name='VWAP 14'),
                  row=1, col=1, secondary_y=False)

    # 4. Plot Dividends on Main Chart
    dividends_df = nvda[nvda['Dividends'] > 0]
    if not dividends_df.empty:
        price_range_for_offset = nvda['High'].max() - nvda['Low'].min()
        if price_range_for_offset == 0: price_range_for_offset = nvda['Low'].mean()
        dividend_y_position = dividends_df['Low'] - (price_range_for_offset * 0.03)

        fig.add_trace(go.Scatter(
            x=dividends_df.index,
            y=dividend_y_position,
            mode='markers',
            marker=dict(
                symbol='triangle-down', # Symbol for dividend
                size=10,
                color=color_dividend,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name='Dividends',
            text=[f'Dividend: {amt:.2f}' for amt in dividends_df['Dividends']],
            hoverinfo='text+x'
        ), row=1, col=1, secondary_y=False)

    # RSI
    fig.add_trace(go.Scatter(x=nvda.index, y=nvda['RSI_14'], mode='lines',
                             line=dict(color=color_rsi14, width=1.5), name='RSI 14'),
                  row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="rgba(255,0,0,0.7)", line_width=1, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="rgba(0,128,0,0.7)", line_width=1, row=2, col=1)


    # Update Layout
    fig.update_layout(
        title=f'{ticker_symbol} - Stock Price, Indicators, and Dividends', # Updated title
        xaxis_title='Date',
        yaxis_title='Price ($)',
        legend_title_text='Legend',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False,
        height=700,
        hovermode='x unified'
    )

    fig.update_yaxes(title_text="Price ($)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Volume", row=1, col=1, secondary_y=True, showticklabels=True,
                     range=[0, nvda['Volume'].max() * 4])
    fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=False)

    fig.show()


elif 'nvda' in locals() and not all(col in nvda.columns for col in required_cols):
    missing_cols = [col for col in required_cols if col not in nvda.columns]
    print(f"Missing required column(s) for plotting:")
    print(f"Available columns in nvda: {nvda.columns.tolist()}")
    if 'Dividends' not in nvda.columns:
        print("Note: 'Dividends' column is missing.")
else:
    print("Stock DataFrame ('nvda') with required indicators not found or is empty.")
    if 'nvda' in locals():
        print(f"Available columns: {nvda.columns.tolist()}")

# SNP 500 ETF

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker_symbol, period, interval):
    print(f"Fetching data for {ticker_symbol} for the period: {period} with interval: {interval}...")
    stock = yf.Ticker(ticker_symbol)
    hist_data = stock.history(period=period, interval=interval)

    return hist_data

ticker_symbol = "SPY"  # Ticker
data_period = "2y"
data_interval = "1d"

# Fetching
spy = fetch_stock_data(ticker_symbol, period=data_period, interval=data_interval)

Fetching data for SPY for the period: 2y with interval: 1d...


## APPLYING TECHNICAL INDICATOR

In [ ]:
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

sma_20 = SMAIndicator(close=spy["Close"], window=20, fillna=True)
spy['SMA_20'] = sma_20.sma_indicator()

sma_50 = SMAIndicator(close=spy["Close"], window=50, fillna=True)
spy['SMA_50'] = sma_50.sma_indicator()

rsi_14 = RSIIndicator(close=spy["Close"], window=14, fillna=True)
spy['RSI_14'] = rsi_14.rsi()

vwap_indicator = VolumeWeightedAveragePrice(high=spy["High"],
                                                low=spy["Low"],
                                                close=spy["Close"],
                                                volume=spy["Volume"],
                                                window=14,
                                                fillna=True)
spy['VWAP_14'] = vwap_indicator.volume_weighted_average_price()

## PLOT SNP500 ETF

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

required_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20', 'SMA_50', 'RSI_14', 'VWAP_14', 'Dividends']
if 'spy' in locals() and not spy.empty and all(col in spy.columns for col in required_cols):

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, # Space between RSI and main chart
                        row_heights=[0.7, 0.3],
                        specs=[[{"secondary_y": True}], # Main chart with secondary y-axis for volume
                               [{"secondary_y": False}]] # RSI chart
                      )

    # 1. Candlestick Chart
    fig.add_trace(go.Candlestick(x=spy.index,
                                 open=spy['Open'],
                                 high=spy['High'],
                                 low=spy['Low'],
                                 close=spy['Close'],
                                 name='Candlestick'),
                  row=1, col=1, secondary_y=False)

    # 2. Volume Bars
    fig.add_trace(go.Bar(x=spy.index, y=spy['Volume'], name='Volume',
                         marker_color='rgba(128,128,128,0.5)'), # Grey color for volume
                  row=1, col=1, secondary_y=True)

    # 3. Add Technical Indicators to Chart
    color_sma20 = 'blue'
    color_sma50 = 'orange'
    color_vwap14 = 'green'
    color_rsi14 = 'brown'
    color_dividend = 'purple' # Color for dividend markers

    fig.add_trace(go.Scatter(x=spy.index, y=spy['SMA_20'], mode='lines',
                             line=dict(color=color_sma20, width=1.5), name='SMA 20'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=spy.index, y=spy['SMA_50'], mode='lines',
                             line=dict(color=color_sma50, width=1.5), name='SMA 50'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=spy.index, y=spy['VWAP_14'], mode='lines',
                             line=dict(color=color_vwap14, width=1.5), name='VWAP 14'),
                  row=1, col=1, secondary_y=False)

    # 4. Plot Dividends on Main Chart
    dividends_df = spy[spy['Dividends'] > 0]
    if not dividends_df.empty:
        price_range_for_offset = spy['High'].max() - spy['Low'].min()
        if price_range_for_offset == 0: price_range_for_offset = spy['Low'].mean()
        dividend_y_position = dividends_df['Low'] - (price_range_for_offset * 0.03)

        fig.add_trace(go.Scatter(
            x=dividends_df.index,
            y=dividend_y_position,
            mode='markers',
            marker=dict(
                symbol='triangle-down', # Symbol for dividend
                size=10,
                color=color_dividend,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name='Dividends',
            text=[f'Dividend: {amt:.2f}' for amt in dividends_df['Dividends']],
            hoverinfo='text+x'
        ), row=1, col=1, secondary_y=False)

    # RSI
    fig.add_trace(go.Scatter(x=spy.index, y=spy['RSI_14'], mode='lines',
                             line=dict(color=color_rsi14, width=1.5), name='RSI 14'),
                  row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="rgba(255,0,0,0.7)", line_width=1, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="rgba(0,128,0,0.7)", line_width=1, row=2, col=1)


    # Update Layout
    fig.update_layout(
        title=f'{ticker_symbol} - Stock Price, Indicators, and Dividends', # Updated title
        xaxis_title='Date',
        yaxis_title='Price ($)',
        legend_title_text='Legend',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False,
        height=700,
        hovermode='x unified'
    )

    fig.update_yaxes(title_text="Price ($)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Volume", row=1, col=1, secondary_y=True, showticklabels=True,
                     range=[0, spy['Volume'].max() * 4])
    fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=False)

    fig.show()


elif 'spy' in locals() and not all(col in spy.columns for col in required_cols):
    missing_cols = [col for col in required_cols if col not in spy.columns]
    print(f"Missing required column(s) for plotting:")
    print(f"Available columns in spy: {spy.columns.tolist()}")
    if 'Dividends' not in spy.columns:
        print("Note: 'Dividends' column is missing.")
else:
    print("Stock DataFrame ('spy') with required indicators not found or is empty.")
    if 'spy' in locals():
        print(f"Available columns: {spy.columns.tolist()}")

# GOOGL

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker_symbol, period, interval):
    print(f"Fetching data for {ticker_symbol} for the period: {period} with interval: {interval}...")
    stock = yf.Ticker(ticker_symbol)
    hist_data = stock.history(period=period, interval=interval)

    return hist_data

ticker_symbol = "GOOGL"  # Ticker
data_period = "5y"
data_interval = "1d"

# Fetching
googl = fetch_stock_data(ticker_symbol, period=data_period, interval=data_interval)

Fetching data for GOOGL for the period: 5y with interval: 1d...


## APPLYING TECHNICAL INDICATOR

In [ ]:
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

sma_20 = SMAIndicator(close=googl["Close"], window=20, fillna=True)
googl['SMA_20'] = sma_20.sma_indicator()

sma_50 = SMAIndicator(close=googl["Close"], window=50, fillna=True)
googl['SMA_50'] = sma_50.sma_indicator()

rsi_14 = RSIIndicator(close=googl["Close"], window=14, fillna=True)
googl['RSI_14'] = rsi_14.rsi()

vwap_indicator = VolumeWeightedAveragePrice(high=googl["High"],
                                                low=googl["Low"],
                                                close=googl["Close"],
                                                volume=googl["Volume"],
                                                window=14,
                                                fillna=True)
googl['VWAP_14'] = vwap_indicator.volume_weighted_average_price()

## PLOT GOOGL

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

required_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20', 'SMA_50', 'RSI_14', 'VWAP_14', 'Dividends']
if 'googl' in locals() and not googl.empty and all(col in googl.columns for col in required_cols):

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, # Space between RSI and main chart
                        row_heights=[0.7, 0.3],
                        specs=[[{"secondary_y": True}], # Main chart with secondary y-axis for volume
                               [{"secondary_y": False}]] # RSI chart
                      )

    # 1. Candlestick Chart
    fig.add_trace(go.Candlestick(x=googl.index,
                                 open=googl['Open'],
                                 high=googl['High'],
                                 low=googl['Low'],
                                 close=googl['Close'],
                                 name='Candlestick'),
                  row=1, col=1, secondary_y=False)

    # 2. Volume Bars
    fig.add_trace(go.Bar(x=googl.index, y=googl['Volume'], name='Volume',
                         marker_color='rgba(128,128,128,0.5)'), # Grey color for volume
                  row=1, col=1, secondary_y=True)

    # 3. Add Technical Indicators to Chart
    color_sma20 = 'blue'
    color_sma50 = 'orange'
    color_vwap14 = 'green'
    color_rsi14 = 'brown'
    color_dividend = 'purple' # Color for dividend markers

    fig.add_trace(go.Scatter(x=googl.index, y=googl['SMA_20'], mode='lines',
                             line=dict(color=color_sma20, width=1.5), name='SMA 20'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=googl.index, y=googl['SMA_50'], mode='lines',
                             line=dict(color=color_sma50, width=1.5), name='SMA 50'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=googl.index, y=googl['VWAP_14'], mode='lines',
                             line=dict(color=color_vwap14, width=1.5), name='VWAP 14'),
                  row=1, col=1, secondary_y=False)

    # 4. Plot Dividends on Main Chart
    dividends_df = googl[googl['Dividends'] > 0]
    if not dividends_df.empty:
        price_range_for_offset = googl['High'].max() - googl['Low'].min()
        if price_range_for_offset == 0: price_range_for_offset = googl['Low'].mean()
        dividend_y_position = dividends_df['Low'] - (price_range_for_offset * 0.03)

        fig.add_trace(go.Scatter(
            x=dividends_df.index,
            y=dividend_y_position,
            mode='markers',
            marker=dict(
                symbol='triangle-down', # Symbol for dividend
                size=10,
                color=color_dividend,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name='Dividends',
            text=[f'Dividend: {amt:.2f}' for amt in dividends_df['Dividends']],
            hoverinfo='text+x'
        ), row=1, col=1, secondary_y=False)

    # RSI
    fig.add_trace(go.Scatter(x=googl.index, y=googl['RSI_14'], mode='lines',
                             line=dict(color=color_rsi14, width=1.5), name='RSI 14'),
                  row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="rgba(255,0,0,0.7)", line_width=1, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="rgba(0,128,0,0.7)", line_width=1, row=2, col=1)


    # Update Layout
    fig.update_layout(
        title=f'{ticker_symbol} - Stock Price, Indicators, and Dividends', # Updated title
        xaxis_title='Date',
        yaxis_title='Price ($)',
        legend_title_text='Legend',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False,
        height=700,
        hovermode='x unified'
    )

    fig.update_yaxes(title_text="Price ($)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Volume", row=1, col=1, secondary_y=True, showticklabels=True,
                     range=[0, googl['Volume'].max() * 4])
    fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=False)

    fig.show()


elif 'googl' in locals() and not all(col in googl.columns for col in required_cols):
    missing_cols = [col for col in required_cols if col not in googl.columns]
    print(f"Missing required column(s) for plotting:")
    print(f"Available columns in googl: {googl.columns.tolist()}")
    if 'Dividends' not in googl.columns:
        print("Note: 'Dividends' column is missing.")
else:
    print("Stock DataFrame ('googl') with required indicators not found or is empty.")
    if 'googl' in locals():
        print(f"Available columns: {googl.columns.tolist()}")

# AMZN

In [ ]:
import yfinance as yf
import pandas as pd

def fetch_stock_data(ticker_symbol, period, interval):
    print(f"Fetching data for {ticker_symbol} for the period: {period} with interval: {interval}...")
    stock = yf.Ticker(ticker_symbol)
    hist_data = stock.history(period=period, interval=interval)

    return hist_data

ticker_symbol = "AMZN"  # Ticker
data_period = "5y"
data_interval = "1d"

# Fetching
amzn = fetch_stock_data(ticker_symbol, period=data_period, interval=data_interval)

Fetching data for AMZN for the period: 5y with interval: 1d...


## APPLYING TECHNICAL INDICATOR

In [ ]:
from ta.trend import SMAIndicator, EMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

sma_20 = SMAIndicator(close=amzn["Close"], window=20, fillna=True)
amzn['SMA_20'] = sma_20.sma_indicator()

sma_50 = SMAIndicator(close=amzn["Close"], window=50, fillna=True)
amzn['SMA_50'] = sma_50.sma_indicator()

rsi_14 = RSIIndicator(close=amzn["Close"], window=14, fillna=True)
amzn['RSI_14'] = rsi_14.rsi()

vwap_indicator = VolumeWeightedAveragePrice(high=amzn["High"],
                                                low=amzn["Low"],
                                                close=amzn["Close"],
                                                volume=amzn["Volume"],
                                                window=14,
                                                fillna=True)
amzn['VWAP_14'] = vwap_indicator.volume_weighted_average_price()

## PLOT AMZN

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

required_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20', 'SMA_50', 'RSI_14', 'VWAP_14', 'Dividends']
if 'amzn' in locals() and not amzn.empty and all(col in amzn.columns for col in required_cols):

    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.05, # Space between RSI and main chart
                        row_heights=[0.7, 0.3],
                        specs=[[{"secondary_y": True}], # Main chart with secondary y-axis for volume
                               [{"secondary_y": False}]] # RSI chart
                      )

    # 1. Candlestick Chart
    fig.add_trace(go.Candlestick(x=amzn.index,
                                 open=amzn['Open'],
                                 high=amzn['High'],
                                 low=amzn['Low'],
                                 close=amzn['Close'],
                                 name='Candlestick'),
                  row=1, col=1, secondary_y=False)

    # 2. Volume Bars
    fig.add_trace(go.Bar(x=amzn.index, y=amzn['Volume'], name='Volume',
                         marker_color='rgba(128,128,128,0.5)'), # Grey color for volume
                  row=1, col=1, secondary_y=True)

    # 3. Add Technical Indicators to Chart
    color_sma20 = 'blue'
    color_sma50 = 'orange'
    color_vwap14 = 'green'
    color_rsi14 = 'brown'
    color_dividend = 'purple' # Color for dividend markers

    fig.add_trace(go.Scatter(x=amzn.index, y=amzn['SMA_20'], mode='lines',
                             line=dict(color=color_sma20, width=1.5), name='SMA 20'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=amzn.index, y=amzn['SMA_50'], mode='lines',
                             line=dict(color=color_sma50, width=1.5), name='SMA 50'),
                  row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(x=amzn.index, y=amzn['VWAP_14'], mode='lines',
                             line=dict(color=color_vwap14, width=1.5), name='VWAP 14'),
                  row=1, col=1, secondary_y=False)

    # 4. Plot Dividends on Main Chart
    dividends_df = amzn[amzn['Dividends'] > 0]
    if not dividends_df.empty:
        price_range_for_offset = amzn['High'].max() - amzn['Low'].min()
        if price_range_for_offset == 0: price_range_for_offset = amzn['Low'].mean()
        dividend_y_position = dividends_df['Low'] - (price_range_for_offset * 0.03)

        fig.add_trace(go.Scatter(
            x=dividends_df.index,
            y=dividend_y_position,
            mode='markers',
            marker=dict(
                symbol='triangle-down', # Symbol for dividend
                size=10,
                color=color_dividend,
                line=dict(width=1, color='DarkSlateGrey')
            ),
            name='Dividends',
            text=[f'Dividend: {amt:.2f}' for amt in dividends_df['Dividends']],
            hoverinfo='text+x'
        ), row=1, col=1, secondary_y=False)

    # RSI
    fig.add_trace(go.Scatter(x=amzn.index, y=amzn['RSI_14'], mode='lines',
                             line=dict(color=color_rsi14, width=1.5), name='RSI 14'),
                  row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="rgba(255,0,0,0.7)", line_width=1, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="rgba(0,128,0,0.7)", line_width=1, row=2, col=1)


    # Update Layout
    fig.update_layout(
        title=f'{ticker_symbol} - Stock Price, Indicators, and Dividends', # Updated title
        xaxis_title='Date',
        yaxis_title='Price ($)',
        legend_title_text='Legend',
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        xaxis_rangeslider_visible=False,
        height=700,
        hovermode='x unified'
    )

    fig.update_yaxes(title_text="Price ($)", row=1, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Volume", row=1, col=1, secondary_y=True, showticklabels=True,
                     range=[0, amzn['Volume'].max() * 4])
    fig.update_yaxes(title_text="RSI", row=2, col=1, secondary_y=False)

    fig.show()


elif 'amzn' in locals() and not all(col in amzn.columns for col in required_cols):
    missing_cols = [col for col in required_cols if col not in amzn.columns]
    print(f"Missing required column(s) for plotting:")
    print(f"Available columns in amzn: {amzn.columns.tolist()}")
    if 'Dividends' not in amzn.columns:
        print("Note: 'Dividends' column is missing.")
else:
    print("Stock DataFrame ('amzn') with required indicators not found or is empty.")
    if 'amzn' in locals():
        print(f"Available columns: {amzn.columns.tolist()}")

# PERFORMANCE STOCK WITH SPY ETF

In [ ]:
import plotly.graph_objects as go
import pandas as pd
date_rng = pd.date_range(end=pd.Timestamp.now(), periods=5*252, freq='B')
sample_data = {
    'Open': [i + (i*0.1* (j%3-1)) for j,i in enumerate(range(100, 100 + len(date_rng)))],
    'High': [i + (i*0.2* (j%3-1)) for j,i in enumerate(range(100, 100 + len(date_rng)))],
    'Low': [i - (i*0.2* (j%3-1)) for j,i in enumerate(range(100, 100 + len(date_rng)))],
    'Close': [i + (i*0.05* (j%2-0.5)) for j,i in enumerate(range(100, 100 + len(date_rng)))]
}
base_close = pd.Series(sample_data['Close'], index=date_rng)

# Create slightly varied data for each ticker for plotting purposes
if 'amzn' not in locals():
    amzn = pd.DataFrame({'Close': base_close * 1.2 + pd.Series(range(len(date_rng)), index=date_rng)*0.1 + 20 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'googl' not in locals():
    googl = pd.DataFrame({'Close': base_close * 1.1 + pd.Series(range(len(date_rng)), index=date_rng)*0.05 + 15 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'nvda' not in locals():
    nvda = pd.DataFrame({'Close': base_close * 2.5 + pd.Series(range(len(date_rng)), index=date_rng)*0.5 + 50 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'aapl' not in locals():
    aapl = pd.DataFrame({'Close': base_close * 1.5 + pd.Series(range(len(date_rng)), index=date_rng)*0.2 + 30 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'msft' not in locals():
    msft = pd.DataFrame({'Close': base_close * 1.3 + pd.Series(range(len(date_rng)), index=date_rng)*0.15 + 25 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'spy' not in locals():
    spy = pd.DataFrame({'Close': base_close * 1.0 + pd.Series(range(len(date_rng)), index=date_rng)*0.02 + 10 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
# --- End of Sample Data Creation ---

stocks_to_plot = {
    "AMZN": amzn,
    "GOOGL": googl,
    "NVDA": nvda,
    "AAPL": aapl,
    "MSFT": msft,
    "SPY": spy
}

fig = go.Figure()

print("Calculating percentage change and plotting...")

for ticker_name, stock_df in stocks_to_plot.items():
    if stock_df is not None and not stock_df.empty and 'Close' in stock_df.columns:
        # Calculate percentage change relative to the first 'Close' price in the DataFrame
        initial_price = stock_df['Close'].iloc[0]
        percentage_change = ((stock_df['Close'] / initial_price) - 1) * 100

        line_style = dict()
        if ticker_name == "SPY":
            line_style = dict(color='black', width=2.5, dash='dash') # Distinct style for SPY

        fig.add_trace(go.Scatter(
            x=stock_df.index,
            y=percentage_change,
            mode='lines',
            name=ticker_name,
            line=line_style if line_style else None # Apply style if it's SPY
        ))
    else:
        print(f"Warning: DataFrame for {ticker_name} is None, empty, or missing 'Close' column. Skipping.")

# --- Update Layout ---
fig.update_layout(
    title='5-Year Stock Performance Comparison (Percentage Change)',
    xaxis_title='Date',
    yaxis_title='Percentage Change (%)',
    legend_title_text='Tickers',
    hovermode='x unified', # Shows all data for a given x (date)
    height=600
)

fig.show()

Calculating percentage change and plotting...


# PERFORMANCE STOCK EXCLUDING NVIDIA

In [ ]:
import plotly.graph_objects as go
import pandas as pd
date_rng = pd.date_range(end=pd.Timestamp.now(), periods=5*252, freq='B')
sample_data = {
    'Open': [i + (i*0.1* (j%3-1)) for j,i in enumerate(range(100, 100 + len(date_rng)))],
    'High': [i + (i*0.2* (j%3-1)) for j,i in enumerate(range(100, 100 + len(date_rng)))],
    'Low': [i - (i*0.2* (j%3-1)) for j,i in enumerate(range(100, 100 + len(date_rng)))],
    'Close': [i + (i*0.05* (j%2-0.5)) for j,i in enumerate(range(100, 100 + len(date_rng)))]
}
base_close = pd.Series(sample_data['Close'], index=date_rng)

# Create slightly varied data for each ticker for plotting purposes
if 'amzn' not in locals():
    amzn = pd.DataFrame({'Close': base_close * 1.2 + pd.Series(range(len(date_rng)), index=date_rng)*0.1 + 20 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'googl' not in locals():
    googl = pd.DataFrame({'Close': base_close * 1.1 + pd.Series(range(len(date_rng)), index=date_rng)*0.05 + 15 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'aapl' not in locals():
    aapl = pd.DataFrame({'Close': base_close * 1.5 + pd.Series(range(len(date_rng)), index=date_rng)*0.2 + 30 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'msft' not in locals():
    msft = pd.DataFrame({'Close': base_close * 1.3 + pd.Series(range(len(date_rng)), index=date_rng)*0.15 + 25 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
if 'spy' not in locals():
    spy = pd.DataFrame({'Close': base_close * 1.0 + pd.Series(range(len(date_rng)), index=date_rng)*0.02 + 10 * pd.Series(np.random.randn(len(date_rng)), index=date_rng)}, index=date_rng)
# --- End of Sample Data Creation ---

stocks_to_plot = {
    "AMZN": amzn,
    "GOOGL": googl,
    "AAPL": aapl,
    "MSFT": msft,
    "SPY": spy
}

fig = go.Figure()

print("Calculating percentage change and plotting...")

for ticker_name, stock_df in stocks_to_plot.items():
    if stock_df is not None and not stock_df.empty and 'Close' in stock_df.columns:
        # Calculate percentage change relative to the first 'Close' price in the DataFrame
        initial_price = stock_df['Close'].iloc[0]
        percentage_change = ((stock_df['Close'] / initial_price) - 1) * 100

        line_style = dict()
        if ticker_name == "SPY":
            line_style = dict(color='black', width=2.5, dash='dash') # Distinct style for SPY

        fig.add_trace(go.Scatter(
            x=stock_df.index,
            y=percentage_change,
            mode='lines',
            name=ticker_name,
            line=line_style if line_style else None # Apply style if it's SPY
        ))
    else:
        print(f"Warning: DataFrame for {ticker_name} is None, empty, or missing 'Close' column. Skipping.")

# --- Update Layout ---
fig.update_layout(
    title='5-Year Stock Performance Comparison (Percentage Change) Excluding NVIDIA',
    xaxis_title='Date',
    yaxis_title='Percentage Change (%)',
    legend_title_text='Tickers',
    hovermode='x unified', # Shows all data for a given x (date)
    height=600
)

fig.show()

Calculating percentage change and plotting...
